This to check the quality of extraction from cluster calculation

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path

%matplotlib widget

folder_base = Path('Gaelle/Baseline_recording/Purple/Rec2/OpenEphys/')
Struct = 'S1'
Events = 'Spindle' # Spindle 
filename3 = folder_base / f'LFPwake0.npy'
filename4 = folder_base / f'Signal{Struct}.npy'
filename5 = folder_base / f'{Events}properties_{Struct}.csv' #
if Struct=='CA1':
    filename5 = folder_base / f'{Events}properties.csv' #

WakeRem = np.load(filename3, mmap_mode= 'r')
SubstProj = np.load(filename4, mmap_mode= 'r')
Events = pd.read_csv(filename5)

EMG = WakeRem[:,27]#27/11
TraceFilt = SubstProj[:,0]
Proj = SubstProj[:,1]
if Struct == 'S1':
    Trace = WakeRem[:,23]#-WakeRem[:,20]#4/20
elif Struct == 'PFC':
    Trace = WakeRem[:,15]#-WakeRem[:,15]#6/22
elif Struct == 'CA1':
    Trace = -WakeRem[:,17]-WakeRem[:,19]#31/15

Events_peak = Events["peak time"].astype(int)
Events_start = Events["start time"].astype(int)
Events_end = Events["end time"].astype(int)

print(TraceFilt.shape, Proj.shape, Trace.shape)

In [ ]:
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer

combined = np.stack([Trace, TraceFilt, Proj], axis = 1)

app = mkQApp()
sample_rate = 1000.
t_start = 0.
# #create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: Events_peak, 1: Events_start, 2: Events_end}
# #and asign them to some channels each
scatter_channels = {0: [0, 1], 1: [1, 2], 2: [1, 2]}
source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

# #Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

# #create a viewer for signal with TraceViewer
# #connected to the signal source
view1 = TraceViewer(source=source)

# #Parameters can be set in script
# view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'


view1.by_channel_params['ch0', 'gain'] = 0.00015
view1.by_channel_params['ch1', 'gain'] = 0.0001
view1.by_channel_params['ch2', 'gain'] = 0.00005


view1.by_channel_params['ch0', 'offset'] = 0.2
view1.by_channel_params['ch1', 'offset'] = -0.2
view1.by_channel_params['ch2', 'offset'] = -0.2
# #Create the main window that can contain several viewers
# win = MainViewer()
# view1 = TraceViewer.from_numpy(combined, sample_rate, t_start, 'Signals')
win.add_view(view1)

# #Parameters can be set in script
# #view1.params['scale_mode'] = 'same_for_all'
# view1.params['display_labels'] = True

#Run
win.show()
app.exec_()

In [ ]:
filename = folder_base / f'RawDataChannelExtractedDS_{ID_rec}.npy'
filename2 = folder_base / f'LFPwake0_{ID_rec}.npy'
All = np.load(filename, mmap_mode= 'r')
Wake0 = np.load(filename2 , mmap_mode= 'r')

plt.close('all')
EMGw0 = WakeRem[:,6]-WakeRem[:,5]
new_sampling_rate = 1000
times = np.arange(0, EMGw0.size/new_sampling_rate, 1./new_sampling_rate)
# Defining subset
start = 0000000
end = 4000000

tt = times[start:end]
EMGt = EMG[start:end]
EMGw0t = EMGw0[start:end]
plt.plot(times, EMGw0)
plt.show()